## Exemple du fonctionnement de l'algorithme de chiffrement utilisé par SEAL
* SEAL est une librairie de chiffrement homomorphe par Microsoft

https://eprint.iacr.org/2017/224.pdf

* SEAL est basé sur le chiffrement BFV

https://eprint.iacr.org/2012/144.pdf

* Code inspiré par

https://arxiv.org/pdf/1906.07127.pdf

* Implémenté sur le framework Python SageMath

https://www.sagemath.org

In [1]:
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler

In [2]:
# Modulus in the ciphertext space
q = 2^35 - 2^14 + 2^11 + 1
# Modulus in the plaintext space (doit etre au moins 2x plus grand que le nombre a representer)
t = 128
# A power of 2
n = 1024
# The polynomial modulus which specifies the ring R
f = x^n + 1
# The ring Z[x]/f(x)
P.<x> = PolynomialRing(ZZ)
R.<x> = QuotientRing(P, P.ideal(f))
# Error distribution (a truncated discrete Gaussian distribution)
noise_standard_deviation = 3.20
dist = DiscreteGaussianDistributionIntegerSampler(sigma=noise_standard_deviation)
# Quotient on division of q by t
delta = floor(q / t)

In [3]:
def rq(poly):
    coefs = poly.list()
    for i in range(len(coefs)):
        coefs[i] = coefs[i] % q
    return R(coefs)

def rt(poly):
    coefs = poly.list()
    for i in range(len(coefs)):
        coefs[i] = coefs[i] % t
    return R(coefs)

# Cyphertext Space
def Roundq(poly):
    coefs = poly.list()
    for i in range(len(coefs)):
        coefs[i] = coefs[i] % q
        if coefs[i] > q / 2:
            coefs[i] = coefs[i] - q
    return R(coefs)

# Plaintext Space 
def Roundt(poly):
    coefs = poly.list()
    for i in range(len(coefs)):
        coefs[i] = coefs[i] % t
        if coefs[i] > t / 2:
            coefs[i] = coefs[i] - t
    return R(coefs)

In [4]:
assert q % (2*n) == 1
assert delta * t + rt(q) == q

In [5]:
# Generate a random ternary polynomial (coefs are either -1, 0 or 1)
def sample_poly_ternary():
    return R([ZZ.random_element(-1, 2, "uniform") for _ in range(n)])

# Generate a random polynomial with integer coefs from a Gaussian Distribution χ
def sample_poly_normal():
    return R([dist() for _ in range(n)])
    
# Generate a random polynomial with coefs in Z/qZ
def sample_Rq():
    return R([ZZ.random_element(0, q, "uniform") for _ in range(n)])

## Génération de clé
* Dans SEAL, la clé privé est un polynome cyclotomique

In [6]:
# Generate a secret key
def secretKeyGen():
    return sample_poly_ternary()

### Calcule de la clé publique correspondante
$$a \leftarrow R_q, e \leftarrow \chi$$
$$pk = \big(-[a*s+e]_q, a\big)$$

In [7]:
def publicKeyGen(s):
    a = sample_Rq()
    e = sample_poly_normal()
    return [-Roundq(a * s + e), a]

In [8]:
sk = secretKeyGen()
pk = publicKeyGen(sk)

## La fonction de chiffrement

In [9]:
def encrypt(p, m):
    u = sample_poly_ternary()
    e1 = sample_poly_normal()
    e2 = sample_poly_normal()
    return [Roundq(p[0] * u + e1 + delta * m), Roundq(p[1] * u + e2)]

def decrypt(s, c):
    tmp = t * Roundq(c[0] + c[1] * s)
    coefs = tmp.list()
    for i in range(len(coefs)):
        coefs[i] = round(coefs[i] / q)
    return Roundt(R(coefs))

In [10]:
assert decrypt(sk, encrypt(pk, 1)) == 1

## Addition
$$\Big(\big[c_1[0] + c_2[0]\big]_q , \big[c_1[1] + c_2[1]\big]_q \Big)$$

In [11]:
def add(c1, c2):
    return [Roundq(c1[0] + c2[0]), Roundq(c1[1] + c2[1])]

### Test
Ajoutons 20 à 22

In [12]:
c1 = encrypt(pk, 20)
c2 = encrypt(pk, 22)
print(decrypt(sk, add(c1, c2)))

42


## TODO: la multiplication
L'algorithme de multiplication de BFV est beaucoup plus compliqué que l'addition.
Il se fait en plusieurs étapes et nécessite des changements de bases.

Voir chapitre 4 de [Somewhat Practical Fully Homomorphic Encryption](https://eprint.iacr.org/2012/144.pdf) 

### Exemple d'une clé privée

In [13]:
print(sk)

x^1023 - x^1022 - x^1020 + x^1019 - x^1018 + x^1015 - x^1013 + x^1011 - x^1007 + x^1006 - x^1005 + x^1003 - x^1001 + x^1000 + x^999 - x^998 + x^997 - x^996 - x^995 - x^993 - x^992 + x^991 + x^988 - x^987 + x^986 - x^985 + x^984 + x^983 - x^982 - x^981 - x^975 + x^974 + x^973 + x^972 + x^971 - x^970 - x^968 - x^966 + x^965 - x^963 - x^961 + x^960 - x^959 - x^958 + x^957 - x^955 + x^953 + x^952 - x^951 + x^950 - x^949 + x^947 + x^946 - x^945 - x^944 - x^942 + x^940 + x^939 - x^938 - x^934 + x^929 - x^925 - x^924 - x^923 + x^922 - x^921 - x^920 + x^919 + x^918 + x^917 - x^916 - x^913 - x^911 - x^910 + x^909 - x^905 - x^904 + x^903 + x^900 + x^899 + x^898 - x^897 + x^896 - x^895 + x^894 - x^892 + x^891 - x^890 + x^888 + x^887 - x^885 + x^884 - x^883 - x^882 + x^881 - x^879 + x^878 - x^877 + x^876 - x^875 - x^873 + x^872 - x^870 + x^869 - x^868 + x^867 + x^863 - x^861 + x^860 - x^859 + x^858 - x^857 + x^856 - x^855 - x^854 + x^853 - x^851 - x^848 - x^847 - x^845 - x^841 - x^840 - x^839 + x^

### Exemple d'un nombre chiffré

In [14]:
print(c1)

[17147250508*x^1023 - 756716766*x^1022 - 15824696430*x^1021 - 16610575543*x^1020 - 1247964411*x^1019 + 14113192055*x^1018 + 11059676589*x^1017 - 8969920995*x^1016 - 13374278241*x^1015 - 7487302848*x^1014 + 2105390508*x^1013 + 1798006795*x^1012 - 346658706*x^1011 - 4053231504*x^1010 + 3344180689*x^1009 - 14973554540*x^1008 - 4088099345*x^1007 - 4860607201*x^1006 + 5663295779*x^1005 - 5101570510*x^1004 + 3366377089*x^1003 - 12468570071*x^1002 - 6042201428*x^1001 + 2266697384*x^1000 - 10738858975*x^999 - 11490152629*x^998 - 5750535948*x^997 - 12215170790*x^996 - 3321700887*x^995 + 16403279469*x^994 - 7792956074*x^993 + 4925608632*x^992 + 10481671646*x^991 - 16460862456*x^990 - 15634542189*x^989 - 15361973018*x^988 - 10196870821*x^987 - 11390992497*x^986 + 3744285145*x^985 + 160740386*x^984 - 1813799932*x^983 - 4001972037*x^982 + 6511490547*x^981 - 9915816359*x^980 - 13757552815*x^979 + 13566629350*x^978 + 16142520831*x^977 + 10674215766*x^976 + 11537210211*x^975 - 2605937722*x^974 + 98900